In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time

# Turn on Copy-on-write mode to addreww view vs copy issues
pd.options.mode.copy_on_write = True 

!python --version

Python 3.12.2


In [2]:
# source file location: 
source_files = 'source_files/median_housing_values/'

file_DP04       = source_files + 'DP04/ACSDP5Y2022.DP04-Data.csv'
file_B25075     = source_files + 'B25075/ACSDT5Y2022.B25075-Data.csv'
file_B25077     = source_files + 'B25077/ACSDT5Y2022.B25077-Data.csv'
file_B25085     = source_files + 'B25085/ACSDT5Y2022.B25085-Data.csv'
file_B25086     = source_files + 'B25086/ACSDT5Y2022.B25086-Data.csv'
file_B992519    = source_files + 'B992519/ACSDT5Y2022.B992519-Data.csv'
file_B992520    = source_files + 'B992520/ACSDT5Y2022.B992520-Data.csv'
file_S2503      = source_files + 'S2503/ACSST5Y2022.S2503-Data.csv'
file_S2506      = source_files + 'S2506/ACSST5Y2022.S2506-Data.csv'
file_S2507      = source_files + 'S2507/ACSST5Y2022.S2507-Data.csv'

file_B25077     = source_files + 'B25077/B25077-Data.csv'



### Read and clean median home prices per county from US Census

- [Housing data](https://www.census.gov/library/video/2023/adrm/downloading-exporting-and-sharing-2020-census-demographic-and-housing-characteristics-data.html) query for appropriate files at US Census
-  US Census - American Community Survey
    - Dataset: B25077 contains Median Value (Dollars)

Note:  The US Census files contained multiple files.  It was necessary to download unzip them first and then read the appropriate file.

In [3]:
# US Census - American Community Survey
# File: B25077 contains Median Value (Dollars)

# create file path
file_B25077     = source_files + 'B25077/ACSDT5Y2022.B25077-Data.csv'

# Read file B25077
# Skip first row as it contains bad data.
data_B25077 = pd.read_csv(   file_B25077, 
                            skiprows=[1],
                            usecols=range(4),
#                            dtype=dtype_dict,
                            low_memory=False
                          )
#data_B25077
# Check value for zipcode 80129

data_B25077.rename(columns={"NAME": "Zipcode", "B25077_001E": "Median Value", "B25077_001M": "Margin of Error"}, inplace=True)

data_B25077[data_B25077['Zipcode'] == 'ZCTA5 80129']

data_B25077

,GEO_ID,Zipcode,Median Value,Margin of Error
0,860Z200US00601,ZCTA5 00601,90900,5504
1,860Z200US00602,ZCTA5 00602,98300,4589
2,860Z200US00603,ZCTA5 00603,129200,7400
3,860Z200US00606,ZCTA5 00606,104700,12971
4,860Z200US00610,ZCTA5 00610,104600,6942
...,...,...,...,...
33769,860Z200US99923,ZCTA5 99923,-,**
33770,860Z200US99925,ZCTA5 99925,284100,32508
33771,860Z200US99926,ZCTA5 99926,163900,17427
33772,860Z200US99927,ZCTA5 99927,-,**


In [4]:
# Clean values
# Drop Rows with "-" in Median Value.
# create a Boolean mask for the rows to remove
mask = data_B25077['Median Value'] == '-'

# select all rows except the ones that contain '-'
data_B25077 = data_B25077[~mask]


In [5]:
data_B25077


,GEO_ID,Zipcode,Median Value,Margin of Error
0,860Z200US00601,ZCTA5 00601,90900,5504
1,860Z200US00602,ZCTA5 00602,98300,4589
2,860Z200US00603,ZCTA5 00603,129200,7400
3,860Z200US00606,ZCTA5 00606,104700,12971
4,860Z200US00610,ZCTA5 00610,104600,6942
...,...,...,...,...
33767,860Z200US99921,ZCTA5 99921,243000,32847
33768,860Z200US99922,ZCTA5 99922,128800,16796
33770,860Z200US99925,ZCTA5 99925,284100,32508
33771,860Z200US99926,ZCTA5 99926,163900,17427


In [6]:
# Remove "ZCTA5 " from Zip Code column
#df['A'] = df['A'].apply(lambda x: x.replace('&amp;', '&'))

data_B25077['Zipcode'] = data_B25077['Zipcode'].apply(lambda x: x.replace('ZCTA5 ', ''))

data_B25077


,GEO_ID,Zipcode,Median Value,Margin of Error
0,860Z200US00601,00601,90900,5504
1,860Z200US00602,00602,98300,4589
2,860Z200US00603,00603,129200,7400
3,860Z200US00606,00606,104700,12971
4,860Z200US00610,00610,104600,6942
...,...,...,...,...
33767,860Z200US99921,99921,243000,32847
33768,860Z200US99922,99922,128800,16796
33770,860Z200US99925,99925,284100,32508
33771,860Z200US99926,99926,163900,17427


#### Note:  Droped ~3400 Zip codes with no values.

## Plot Chloropeth with Zip Codes and median home values



In [14]:
!pip list | grep nbformat

nbformat                  4.2.0


In [15]:
# Needs to be refined.


#df = data_B25077
#df = zipcode_data[['ZIP_Code','County_Name','Cluster']]
import plotly.express as px
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)
# fig = px.choropleth(df, 
#                     geojson=zipcodes, 
#                     locations='ZIP_Code', 
#                     color='Cluster',
#                     color_continuous_scale="Viridis",
#                     range_color=(1,5),
#                     featureidkey="properties.ZCTA5CE10",
#                     scope="usa",
#                     labels={'Cluster':'Cluster_Category'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})



### Example plotting Counties

In [16]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed